In [ ]:
import os
from urllib import parse
from xml import etree
import zipfile

import requests

import numpy as np

from astropy import units as u
from astropy import table
from astropy import wcs
from astropy.coordinates import Distance

from astropy.io import fits

In [ ]:
import glue_jupyter as gj
from glue import core as gcore

# Load the Pleiades 

We start by downloading the Gaia DR3 dataset for the 3 degree area around the Pleiades using `astroquery`

In [ ]:
from astroquery import gaia

For speeding up the query, we are selecting a smaller data set with the same number of sources, but a reduced set of available columns.
To reset to the (default) full catalogue, set to 'gaiadr3.gaia_source' or, for DR2, to 'gaiadr2.gaia_source'
Our query would return some 63,000 results; again for performance increase the default limit of 50 to just 5,000 and further narrow down the list of columns.

In [ ]:
gaia.Gaia.MAIN_GAIA_TABLE = 'gaiadr3.gaia_source_lite'
gaia.Gaia.ROW_LIMIT = 5000
gaia_cols = ['source_id', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error',
             'phot_g_mean_mag' , 'phot_bp_mean_mag', 'phot_rp_mean_mag', 'bp_rp', 'radial_velocity']


The below will either download the dataset and save it locally, or it will load the saved copy if already downloaded.

In [ ]:
tab = gaia.Gaia.query_object_async('Messier 45', radius=90*u.arcmin, columns=gaia_cols)
tab = table.QTable(tab)

Now let's select a sub-set of interesting columns, and sub-sample on the part of the dataset that has a valid distance and is within a reasonable range that could be anywhere near us.

In [ ]:
subtab = tab['source_id', 'ra', 'dec', 'parallax', 'parallax_error', 'pmra', 'pmra_error', 'pmdec', 'pmdec_error', 
             'phot_g_mean_mag' ,'bp_rp']

subtab['distance'] = subtab['parallax'].to(u.pc, u.parallax())
subtab['distance_error'] = subtab['parallax_error'].to(u.pc, u.parallax())

subtab = subtab[np.isfinite(subtab['distance'])&(subtab['distance']<300*u.pc)&(subtab['distance']>0)]
len(subtab)

# Set up Glupyter 

Now we set up a data object for Glupyter, and create a Glupyter app for further examination.

In [ ]:
gaia_data = gcore.Data('Gaia_Pleiades', **subtab)
app = gj.jglue(gaia_data)

In [ ]:
scats = []

In [ ]:
view3d = app.scatter3d(x='ra', y='dec', z='distance')

In [ ]:
scats.append(app.scatter2d(x='distance', y='phot_g_mean_mag'))

In [ ]:
scats.append(app.scatter2d(x='bp_rp', y='phot_g_mean_mag'))

state = scats[-1].state
state.y_max, state.y_min = state.y_min, state.y_max

In [ ]:
scats.append(app.scatter2d(x='pmra', y='pmdec'))
scats[-1].scale_x.min = -100
scats[-1].scale_x.max = 100
scats[-1].scale_y.min = -100
scats[-1].scale_y.max = 100

In [ ]:
scats.append(app.scatter2d(x='pmra', y='pmra_error'))
scats[-1].state.x_min, scats[-1].state.x_max = -50, 50

scats.append(app.scatter2d(x='pmdec', y='pmdec_error'))
scats[-1].state.x_min, scats[-1].state.x_max = -100, 50

Now go ahead a select a subset using any or all of the plots above (e.g. set the combine as 'and' mode for Subset)...

In [ ]:
if len(gaia_data.subsets) == 0:
    print("You didn't select anything before running this cell, "
          "so I'll select the brightest stars for you!")
    app.subset('Bright stars', gaia_data.id['phot_g_mean_mag'] < 8)
print(f"Selected {len(gaia_data.subsets[0].to_index_list())} sources.")

In [ ]:
seltab = subtab[gaia_data.subsets[0].to_mask()]

And compute the distance to the Pleiades!

In [ ]:
np.mean(seltab['distance'])

Or some percentiles

In [ ]:
np.percentile(seltab['distance'], [10, 32, 50, 68, 90])

And while we're at it - lets see what the mean proper motion is:

In [ ]:
np.mean(seltab['pmra']), np.mean(seltab['pmdec'])

In [ ]:
np.mean(np.hypot(seltab['pmra']*np.cos(seltab['dec']), seltab['pmdec']))

# Now overlay HST image 

Now we look at an HST image of a small part of the Pleiades, and try overplotting some of the Gaia stars

In [ ]:
from astroquery.mast import Observations

In [ ]:
mast_qry = Observations.query_criteria(target_name='PLEIADESFIELD1B', obs_collection='hst')
mast_qry

Now lets download just the final drizzled image for the observation with the bluer filter (highest resolution): F475W

In [ ]:
result = Observations.download_products(mast_qry['obsid'][mast_qry['filters']=='F475W'], productSubGroupDescription='DRZ')
result['Local Path']

In [ ]:
assert len(result) == 1

f475w_image = fits.open(result['Local Path'][0])

Now we add the image data to glue and display it:

In [ ]:
sciim = app.add_data(f475w=f475w_image)[0]
imview = app.imshow(data=sciim)

Now we glue together the image and the Gaia data. (Note that it might take some time for the view to catch up after you make the link.)

In [ ]:
app.add_link(gaia_data, 'ra', sciim, 'Right Ascension')
app.add_link(gaia_data,'dec', sciim, 'Declination')

imview.add_data(gaia_data)

Markers should appear in the image viewer. You may need to fiddle with the color a bit to see the markers.  Since you've already made your subset selection: are any of the Pleiades members in the HST field?

You'll probably see them: they are there, but clearly offset from where the HST image has them!  This might at first appear to be a bug, but it's not.  Why might this be?

_This notebook is making use of data from the European Space Agency (ESA) mission [**Gaia**](https://www.cosmos.esa.int/gaia), processed by the Gaia Data Processing and Analysis Consortium ([**DPAC**](https://www.cosmos.esa.int/web/gaia/dpac/consortium)). Funding for the DPAC has been provided by national institutions, in particular the institutions participating in the Gaia Multilateral Agreement._